<a href="https://colab.research.google.com/github/ddekun/NLP/blob/lesson14/lesson14/hw14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку естественного языка

### Урок 14. Transfer learning

In [31]:
!pip install transformers

In [32]:
import pandas as pd
DATASET_PATH = '/content/drive/MyDrive/Geekbrains/Introduction to natural language processing/Lesson14/dataset.jsonl'

with open(DATASET_PATH) as f:
    df = pd.read_json(DATASET_PATH, lines=True).set_index('id')

In [33]:
df.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [34]:
data = df.loc[:5000, 'text']

In [35]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.15)

In [36]:
import re

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\[\w+\]", "", summary)
        summary = re.sub(r"<[\w+,\!, -]>", "", summary)
        summary = re.sub(r"<\w+>", "", summary)
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [37]:
with open('train_dataset.txt') as f:
    train_dataset = f.read()

In [38]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 1197
Test dataset length: 212


In [39]:
from transformers import AutoTokenizer
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [40]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [41]:
from transformers import TextDataset, DataCollatorForLanguageModeling

#создает тренировочный и тестовый датасет
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [42]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
training_args = TrainingArguments(
    output_dir="gdrive/MyDrive/GPT/gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('/content/gpt2-chief')

model.save_pretrained('/content/model_gpt_chf')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/gpt2-chief")

model1 = AutoModelForCausalLM.from_pretrained("/content/model_gpt_chf")

In [ ]:
!pip install corus
!pip install datasets

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

In [ ]:
from corus import load_lenta2

path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

In [ ]:
data = [(record.title, record.text) for record in records]

In [ ]:
import pandas as pd
df_news = pd.DataFrame({'title': [record[0] for record in data], 'text': [record[1] for record in data]})
df_news.head()

In [ ]:
max(len(i) for i in df_news['text'].values), max(len(i) for i in df_news['title'].values)

In [ ]:
data_news = df_news[:500]

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data_news, test_size=0.15)

In [ ]:
from datasets import Dataset
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_train, dataset_test

In [ ]:
from transformers import AutoTokenizer

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len_txt = 400
max_len_tlt = 50

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tlt)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
dataset_train, dataset_test

In [ ]:
dataset_train.save_to_disk('gazeta/train')
dataset_test.save_to_disk('gazeta/test')

In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
model_name = "IlyaGusev/rut5_base_sum_gazeta"
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
output_dir = 'gazeta/output'
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=1000,
    remove_unused_columns=True,
    eval_steps=500
)

In [ ]:
from datasets import load_from_disk
dataset_train = load_from_disk("gazeta/train")
dataset_test = load_from_disk("gazeta/test")
dataset_train, dataset_test

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

In [ ]:
trainer.save_model(output_dir + '/model')

In [ ]:
import torch
#тестовые данные
INX = 10
input_text = dataset_test['text'][INX]
input_title = dataset_test['title'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("Text:\n" + input_text)
print("Real title: " + input_title)
print("Pred title: " + pred)